In [ ]:
# ===== Pacotes / Packages =====
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Fixar seed para reprodutibilidade / Fix seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# ===== Carregar dados .xlsx / Load data .xlsx =====
df = pd.read_excel("NAME.xlsx")
print(df.head())

In [ ]:
# ===== Carregar dados .csv / Load data .csv =====
df = pd.read_csv("NAME.csv", sep=',') # especifica o separador / specifies the separator
print(df.head())

In [ ]:
# Separar features (X) e target (y) / Separate features (X) and target (y)
X = df.iloc[:, 1:].values   # todas as colunas menos a primeira / all columns except the first
y = df.iloc[:, 0].values    # apenas a primeira coluna / just the first column

# Se 'y' contiver strings, codificar os rótulos / If 'y' contains strings, encode the labels
if y.dtype == 'object': # ou use isinstance(y[0], str) se preferir verificar um elemento / or use isinstance(y[0], str) if you prefer to check one element
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
else:
    y_encoded = y

# Transformar y em categorias (one-hot encoding) / Transform y into categories (one-hot encoding)
# Use y_encoded para one-hot encoding / Use y_encoded for one-hot encoding
y_cat = to_categorical(y_encoded)

# Dividir em treino e teste / Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat, test_size=0.2, random_state=42, stratify=y_encoded # stratify com y_encoded
)

# Normalizar dados / Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Evaluation of parameters / avaliação de parametros.

# Reshape data for LSTM / Redimensionar dados para LSTM
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# List of recurrent unit counts to test
recurrent_unit_counts = [32, 64, 128] # Example: Test models with 32, 64, and 128 LSTM units

val_losses = []
test_losses = []
test_accuracies = []

# Keep the number of LSTM layers fixed to 1 for this evaluation
num_fixed_layers = 1

for num_recurrent_units in recurrent_unit_counts:
    print(f"\nTraining model with {num_recurrent_units} LSTM units (in {num_fixed_layers} layer(s))")

    # Define model with varying number of LSTM units
    model_current = Sequential([
        LSTM(num_recurrent_units, input_shape=(X_train_reshaped.shape[1], 1)), # Single LSTM layer
        Dense(32, activation='relu'),
        Dense(32, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_cat.shape[1], activation="softmax")
    ])

    # Compile model
    model_current.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    # EarlyStopping
    early_stop_current = EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )

    # Train model using reshaped data for LSTM
    history_current = model_current.fit(
        X_train_reshaped, y_train,
        validation_split=0.2,
        epochs=50,
        batch_size=16,
        verbose=0, # Set verbose to 0 to avoid cluttering output during hyperparameter search
        callbacks=[early_stop_current]
    )

    # Evaluate model on test set using reshaped data
    loss_current, acc_current = model_current.evaluate(X_test_reshaped, y_test, verbose=0)
    print(f"Model with {num_recurrent_units} LSTM units - Test accuracy: {acc_current:.4f}, Test loss: {loss_current:.4f}")

    # Store the minimum validation loss achieved during training for plotting
    val_losses.append(min(history_current.history['val_loss']))
    test_losses.append(loss_current)
    test_accuracies.append(acc_current)

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(recurrent_unit_counts, val_losses, marker='o', label='Validation Loss')
plt.plot(recurrent_unit_counts, test_losses, marker='x', label='Test Loss')
plt.title('Loss vs. Number of Recurrent Units')
plt.xlabel('Number of Recurrent Units')
plt.ylabel('Loss')
plt.xticks(recurrent_unit_counts) # Ensure x-ticks correspond to the unit counts
plt.legend()
plt.grid(True)
plt.show()

# Find the best scenario
# Using test loss to determine the best model configuration
best_index = np.argmin(test_losses)
best_num_recurrent_units = recurrent_unit_counts[best_index]
best_test_loss = test_losses[best_index]
best_test_accuracy = test_accuracies[best_index]
best_val_loss = val_losses[best_index]

print(f"\nBest Scenario:")
print(f"Number of Recurrent Units: {best_num_recurrent_units}")
print(f"Corresponding Validation Loss (min): {best_val_loss:.4f}")
print(f"Corresponding Test Loss: {best_test_loss:.4f}")
print(f"Corresponding Test Accuracy: {best_test_accuracy:.4f}")


In [ ]:
# Evaluation of parameters / avaliação de parametros.

# Reshape data for LSTM / Redimensionar dados para LSTM
# Ensure X_train_reshaped and X_test_reshaped are available
if 'X_train_reshaped' not in globals():
    X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Fixed number of recurrent units and layers based on user's request
fixed_num_recurrent_units = 32
fixed_num_recurrent_layers = 1 # As per the new request

# List of Dense layer counts to test
dense_layer_counts = [1, 2, 3, 4] # Example range: 1, 2, 3, or 4 Dense layers (excluding the output layer)

val_losses = []
test_losses = []
test_accuracies = []

for num_dense_layers in dense_layer_counts:
    print(f"\nTraining model with {fixed_num_recurrent_layers} LSTM layer(s) ({fixed_num_recurrent_units} units) and {num_dense_layers} Dense layer(s)")

    model_current = Sequential()

    # Add fixed number of LSTM layers
    for i in range(fixed_num_recurrent_layers):
        if i == 0:
            model_current.add(LSTM(fixed_num_recurrent_units, input_shape=(X_train_reshaped.shape[1], 1), return_sequences=(fixed_num_recurrent_layers > 1)))
        elif i < fixed_num_recurrent_layers - 1:
            model_current.add(LSTM(fixed_num_recurrent_units, return_sequences=True))
        else:
            model_current.add(LSTM(fixed_num_recurrent_units))

    # Add varying number of Dense layers
    for _ in range(num_dense_layers):
        model_current.add(Dense(32, activation='relu'))

    # Add the final output layer
    model_current.add(Dense(y_cat.shape[1], activation="softmax"))

    model_current.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    early_stop_current = EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )

    history_current = model_current.fit(
        X_train_reshaped, y_train,
        validation_split=0.2,
        epochs=50,
        batch_size=16,
        verbose=0,
        callbacks=[early_stop_current]
    )

    loss_current, acc_current = model_current.evaluate(X_test_reshaped, y_test, verbose=0)
    print(f"Model with {num_dense_layers} Dense layers - Test accuracy: {acc_current:.4f}, Test loss: {loss_current:.4f}")

    val_losses.append(min(history_current.history['val_loss']))
    test_losses.append(loss_current)
    test_accuracies.append(acc_current)

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(dense_layer_counts, val_losses, marker='o', label='Validation Loss')
plt.plot(dense_layer_counts, test_losses, marker='x', label='Test Loss')
plt.title(f'Loss vs. Number of Dense Layers (Fixed LSTM Units = {fixed_num_recurrent_units}, Fixed LSTM Layers = {fixed_num_recurrent_layers})')
plt.xlabel('Number of Dense Layers')
plt.ylabel('Loss')
plt.xticks(dense_layer_counts)
plt.legend()
plt.grid(True)
plt.show()

best_index = np.argmin(test_losses)
best_num_dense_layers = dense_layer_counts[best_index]
best_test_loss = test_losses[best_index]
best_test_accuracy = test_accuracies[best_index]
best_val_loss = val_losses[best_index]

print(f"\nBest Scenario:")
print(f"Number of Dense Layers: {best_num_dense_layers}")
print(f"Fixed Number of Recurrent Units: {fixed_num_recurrent_units}")
print(f"Fixed Number of Recurrent Layers: {fixed_num_recurrent_layers}")
print(f"Corresponding Validation Loss (min): {best_val_loss:.4f}")
print(f"Corresponding Test Loss: {best_test_loss:.4f}")
print(f"Corresponding Test Accuracy: {best_test_accuracy:.4f}")


In [ ]:
# ===== Definir modelo RNN / Define RNN Model =====

if 'X_train_reshaped' not in locals():
    X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

model = Sequential([
    LSTM(32, input_shape=(X_train_reshaped.shape[1], 1)), # LSTM layer
    Dense(32, activation='relu'), # Dense layer
    Dense(y_cat.shape[1], activation="softmax") # Output layer
])


In [ ]:
# Compilar modelo / compile model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# EarlyStopping
early_stop = EarlyStopping(
    monitor="val_loss",       # monitora o erro de validação / monitors the validation error
    patience=5,               # quantas épocas sem melhora antes de parar / how many epochs without improvement before stopping
    restore_best_weights=True # garante que volte para a melhor época / ensures that it returns to the best season
)

# Treinar modelo / Train model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# Avaliar modelo / Evaluate model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {acc:.4f}")

# Melhor época
melhor_ep = np.argmin(history.history["val_loss"]) + 1
melhor_val = history.history["val_loss"][melhor_ep - 1]
print(f"Best epoch chosen by EarlyStopping: {melhor_ep} (val_loss = {melhor_val:.4f})")


In [ ]:
# ===== Plotar curvas de Loss e Acurácia / Plot Loss and Accuracy Curves =====

# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# ===== Matriz de confusão / Confusion matrix =====
# Previsões / Predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Matriz / Matrix
cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=np.unique(y),
            yticklabels=np.unique(y))
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion matrix")
plt.show()

# ===== Relatório de classificação / classification report =====
print("Classification report:\n")
print(classification_report(y_true, y_pred_classes, target_names=[f"Class {c}" for c in np.unique(y)]))

In [ ]:
# Use curvas individuais (multiclass ROC) quando quiser analisar o desempenho do modelo em cada classe separadamente. / Use curvas individuais (ROC multiclasse) quando quiser analisar o desempenho do modelo em cada classe separadamente.

# Fit LabelEncoder to the original unique classes / Ajustar LabelEncoder às classes originais exclusivas
label_encoder = LabelEncoder()
label_encoder.fit(np.unique(y))

# Compute ROC curve and ROC area for each class / Calcular a curva ROC e a área ROC para cada classe
fpr = dict()
tpr = dict()
roc_auc = dict()

unique_classes = np.unique(y)  # Use original unique classes for labeling / Use classes originais exclusivas para rotulagem
n_classes = len(unique_classes)

for i in range(n_classes):
    class_value = unique_classes[i]
    # Use label_encoder to get the correct index for the current class / Use label_encoder para obter o índice correto para a classe atual
    class_index = label_encoder.transform([class_value])[0]

    # Check if the current class is present in the test set / Verifique se a classe atual está presente no conjunto de teste
    if np.sum(y_test[:, class_index]) > 0:
        fpr[i], tpr[i], _ = roc_curve(y_test[:, class_index], y_pred[:, class_index])
        roc_auc[i] = auc(fpr[i], tpr[i])
    else:
        # If a class is not in the test set, set AUC to NaN and skip plotting its curve / Se uma classe não estiver no conjunto de teste, defina AUC como NaN e pule a plotagem de sua curva
        roc_auc[i] = np.nan
        print(f"Warning: Class {class_value} has no positive samples in the test set. Skipping ROC curve for this class.")


# Plot ROC curves / Traçar curvas ROC
plt.figure(figsize=(8, 6))
colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
for i in range(n_classes):
    # Only plot if the AUC was calculated (class was in the test set) / Somente plote se a AUC foi calculada (a classe estava no conjunto de teste)
    if not np.isnan(roc_auc[i]):
        # Ensure there are enough colors / Certifique-se de que há cores suficientes
        color = colors[i % len(colors)]
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label='ROC curve of class {0} (area = {1:0.2f})'.format(unique_classes[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Use macro-average ROC quando quiser uma visão geral do desempenho do modelo em todas as classes, sem considerar desequilíbrio entre elas. / Use ROC macro-médio quando quiser uma visão geral do desempenho do modelo em todas as classes, sem considerar desequilíbrio entre elas.

# Calculate macro-average ROC curve and AUC / Calcular a curva ROC macromédia e a AUC
# First aggregate all false positive rates / Primeiro, agregue todas as taxas de falsos positivos
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes) if not np.isnan(roc_auc[i])]))

# Then interpolate all ROC curves at this points / Em seguida, interpole todas as curvas ROC nesses pontos
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    if not np.isnan(roc_auc[i]):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Average it and compute AUC / Calcule a média e a AUC
mean_tpr /= sum([not np.isnan(roc_auc[i]) for i in range(n_classes)]) # Divide by the number of classes that were in the test set/ Divida pelo número de classes que estavam no conjunto de teste

macro_roc_auc = auc(all_fpr, mean_tpr)

# Plot macro-average ROC curve
plt.figure(figsize=(8, 6))
plt.plot(all_fpr, mean_tpr, color='red', linestyle='-', linewidth=2,
         label='Macro-average ROC curve (area = {0:0.2f})'.format(macro_roc_auc))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curve (Macro-average)')
plt.legend(loc="lower right")
plt.show()